In [7]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
test_df = pd.read_csv('data.csv')
test_df = test_df.drop(labels=['Unnamed: 0'],axis='columns')


name_cut = []
for d in df['中文名稱']:
    ch =re.compile("[\u4e00-\u9fa5]")
    name =  "".join(ch.findall(d))
    name_cut.append(name)

type_cut = []
for d in df['類型']:
    ch =re.compile("[\u4e00-\u9fa5]")
    seg_word =  "".join(ch.findall(d))
    types = jieba.lcut(seg_word)
    type_cut.append(types)

intro_cut = []
for d in df['簡介']:
    intro = jieba.analyse.extract_tags(d,topK=2)
    intro_cut.append(intro)
    

data = {'name':name_cut,'type':type_cut ,'article':intro_cut}
df1 = pd.DataFrame(data)

df1['type'] = df1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
df1['article'] = df1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
df1['key'] = df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['article'].astype(str)


test_df = df1